# Лабораторная работа №6
## Задание:
1 . Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
### Подключение библиотек


In [1]:
!pip install catboost

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt

### Загрузка данных и вывод датасета


In [2]:
Dataset = pd.read_csv('property_assessment.csv')
print(Dataset.head())

         PID        CM_ID     GIS_ID  ST_NUM       ST_NAME         CITY  \
0  100018006  100018000.0  100018000   239.0  Lexington ST  EAST BOSTON   
1  100018008  100018000.0  100018000   239.0  Lexington ST  EAST BOSTON   
2  100019006  100019000.0  100019000   241.0  Lexington ST  EAST BOSTON   
3  100024002  100024000.0  100024000   249.0  Lexington ST  EAST BOSTON   
4  100041006  100041000.0  100041000   224.0  PRINCETON ST  EAST BOSTON   

   ZIP_CODE  BLDG_SEQ  NUM_BLDGS  LUC  ... FULL_BTH HLF_BTH KITCHENS TT_RMS  \
0    2128.0         1          1  102  ...      2.0     0.0      1.0    5.0   
1    2128.0         1          1  102  ...      2.0     0.0      1.0    4.0   
2    2128.0         1          1  102  ...      2.0     0.0      1.0    4.0   
3    2128.0         1          1  102  ...      1.0     0.0      1.0    5.0   
4    2128.0         1          1  102  ...      2.0     0.0      1.0    5.0   

     KITCHEN_TYPE           HEAT_TYPE         AC_TYPE FIREPLACES  NUM_PARK

### Создание искомого столбца

In [3]:
label_encoder = LabelEncoder()
Dataset['COND_ENC'] = label_encoder.fit_transform(Dataset['OVERALL_COND'])

### Нормализация данных

In [4]:
Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']] = Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']].applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x).apply(pd.to_numeric, errors='coerce')
numeric_columns = Dataset.select_dtypes(include='number').columns.tolist()
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PID          10321 non-null  int64  
 1   CM_ID        10321 non-null  float64
 2   GIS_ID       10321 non-null  int64  
 3   ST_NUM       10321 non-null  float64
 4   ZIP_CODE     10321 non-null  float64
 5   BLDG_SEQ     10321 non-null  int64  
 6   NUM_BLDGS    10321 non-null  int64  
 7   LUC          10321 non-null  int64  
 8   RES_FLOOR    10321 non-null  float64
 9   GROSS_AREA   10321 non-null  float64
 10  LIVING_AREA  10321 non-null  float64
 11  LAND_VALUE   10321 non-null  int64  
 12  BLDG_VALUE   10321 non-null  int64  
 13  SFYI_VALUE   10321 non-null  int64  
 14  TOTAL_VALUE  10321 non-null  int64  
 15  YR_BUILT     10321 non-null  float64
 16  YR_REMODEL   10321 non-null  float64
 17  BED_RMS      10321 non-null  float64
 18  FULL_BTH     10321 non-null  float64
 19  HLF_

### Поиск лучшей метрики и количества соседей

In [5]:
X = Dataset.drop(columns=['COND_ENC'])
y = Dataset['COND_ENC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Параметры для настройки CatBoost
catboost_params = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8]
}

# Создание модели CatBoost
catboost_model = CatBoostClassifier(verbose=False)

# Подбор лучших параметров с помощью GridSearchCV
catboost_grid_search = GridSearchCV(catboost_model, catboost_params, cv=5)
catboost_grid_search.fit(X_train, y_train)

# Лучшие параметры
best_catboost_params = catboost_grid_search.best_params_
print(f"Лучшие параметры для CatBoost: {best_catboost_params}")


Лучшие параметры для CatBoost: {'depth': 8, 'iterations': 300, 'learning_rate': 0.1}


#### Обучение на лучших метриках и количестве соседей

In [6]:
# Переобучение модели с лучшими параметрами на всем тренировочном наборе данных
best_catboost_model = CatBoostClassifier(**best_catboost_params, verbose=False)
best_catboost_model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred_catboost = best_catboost_model.predict(X_test)

# Оценка точности
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f"Точность CatBoost на тестовых данных: {accuracy_catboost}")

# Отчет о классификации для CatBoost
print("Classification Report для CatBoost:")
print(classification_report(y_test, y_pred_catboost))

Точность CatBoost на тестовых данных: 0.9108743037054977
Classification Report для CatBoost:
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      3050
           1       0.88      0.69      0.77        62
           2       0.88      0.75      0.81        60
           3       1.00      1.00      1.00         1
           4       0.83      0.79      0.81       915
           5       0.93      0.63      0.75        41

    accuracy                           0.91      4129
   macro avg       0.91      0.80      0.85      4129
weighted avg       0.91      0.91      0.91      4129

